In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
# load in the data
X_train = np.load('/content/gdrive/My Drive/training_feature_arr_MLP.npy')
y_train = np.load('/content/gdrive/My Drive/training_labels_MLP.npy')
X_test = np.load('/content/gdrive/My Drive/testing_feature_arr_MLP.npy')
y_test = np.load('/content/gdrive/My Drive/testing_labels_MLP.npy')

In [0]:
# use LASSO technique to do some feature selection because there are 170 features
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

# need to use the one with categories (not one hot encoded)
# to do feature selection

# read in the csv file and put it in a DataFrame.
df_train = pd.read_csv('/content/gdrive/My Drive/audio_training_data_cleaned.csv')

# drop any null values we may have forgotten
df_train = df_train.dropna(how='any',axis=0)

y_train_classes = df_train['age']

# to do feature selection, must convert categorical values to
# numerical values
replaced = {'teens':1,'twenties':2,'thirties':3,'fourties':4,'fifties':5,'sixties':6,
            'seventies':7,'eighties':8}
y_train_encoded = y_train_classes.replace(replaced)



lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train_encoded)
model = SelectFromModel(lsvc, prefit=True)
X_train = model.transform(X_train)

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
# MLP model in Keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.regularizers import l2

DL_model = Sequential()

DL_model.add(Dense(256, activation='relu'))
DL_model.add(Dropout(0.2))

DL_model.add(Dense(1024, activation='relu'))

DL_model.add(Dense(1024, activation='relu'))
DL_model.add(Dropout(0.2))

DL_model.add(Dense(256, activation='relu'))

DL_model.add(Dense(8, activation='softmax'))

DL_model.compile(optimizer='adam',loss='categorical_crossentropy',
             metrics=['accuracy'])
DL_model.fit(X_train,y_train,epochs=50,batch_size=100,class_weight='balanced')

Using TensorFlow backend.


Epoch 1/50
73765/73765 [==============================] - 8s 112us/step - loss: 1.7226 - acc: 0.3158
Epoch 2/50
73765/73765 [==============================] - 7s 101us/step - loss: 1.5970 - acc: 0.3630
Epoch 3/50
73765/73765 [==============================] - 7s 101us/step - loss: 1.5146 - acc: 0.4046
Epoch 4/50
73765/73765 [==============================] - 7s 100us/step - loss: 1.4281 - acc: 0.4386
Epoch 5/50
73765/73765 [==============================] - 7s 100us/step - loss: 1.3344 - acc: 0.4789
Epoch 6/50
73765/73765 [==============================] - 7s 99us/step - loss: 1.2515 - acc: 0.5175
Epoch 7/50
73765/73765 [==============================] - 7s 100us/step - loss: 1.1725 - acc: 0.5525
Epoch 8/50
73765/73765 [==============================] - 7s 99us/step - loss: 1.1076 - acc: 0.5784
Epoch 9/50
73765/73765 [==============================] - 7s 99us/step - loss: 1.0464 - acc: 0.6060
Epoch 10/50
73765/73765 [==============================] - 7s 99us/step - loss: 0.9912 - acc: 

In [0]:
X_test = model.transform(X_test)
DL_model.save('MLP_Model2.h5')
from google.colab import files

files.download('MLP_Model2.h5')

# need to export X_train 
# with the selected features
np.save('training_feature_arr_MLP_Model2.npy',X_train)
files.download('training_feature_arr_MLP_Model2.npy')

# need to export X_test with
# the selected features
np.save('testing_feature_arr_MLP_Model2.npy',X_test)
files.download('testing_feature_arr_MLP_Model2.npy')